# Entity Matching using Neo4j - Error analysis

_Salomon Tetelepta, April 27th 2024_
* Notebook to dig into the errors that remain after entity matching using a graph representation of the Abt-Buy Dataset

### Install dependencies

In [1]:
!pip install neo4j python-dotenv langchain-community --quiet

%load_ext watermark
%watermark -p neo4j

neo4j: 5.17.0



### Imports

In [50]:
from dotenv import load_dotenv, find_dotenv, dotenv_values
from langchain_community.graphs import Neo4jGraph
from pathlib import Path
from sklearn.manifold import TSNE
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from wordfreq import lossy_tokenize, tokenize, word_frequency

import json
import matplotlib.pyplot as plt
import neo4j
import numpy as np

import os
import pandas as pd
import pickle
import re

### Settings

In [3]:
# path settings
project_path = Path(os.getcwd()).parent
data_path = project_path / "data"
output_path = project_path / "output"

database = "abt-buy"

# load env settings
load_dotenv(project_path / ".env")

# reproducability
np.random.seed(42)

### 1. Load Data

In [4]:
os.listdir(data_path / 'abt-buy' / 'record_descriptions')

['2_buy.csv', '1_abt.csv']

In [5]:
# abt and buy records
df_abt = pd.read_csv(data_path / 'abt-buy' / 'record_descriptions' / '1_abt.csv', encoding='unicode_escape')
df_buy = pd.read_csv(data_path / 'abt-buy' / 'record_descriptions' / '2_buy.csv')

# matches - train and validation set
df_train = pd.read_csv(data_path / 'abt-buy' / 'gs_train.csv')
df_val = pd.read_csv(data_path / 'abt-buy' / 'gs_val.csv')
df_test = pd.read_csv(data_path / 'abt-buy' / 'gs_test.csv')

# merge records with matches
df_abt_merged = df_abt.merge(df_train, left_on='subject_id', right_on='source_id', how='right')
df_train_merged = df_buy.merge(df_abt_merged, left_on='subject_id', right_on='target_id', how='right')

df_train_merged.head(3)

,subject_id_x,name_x,description_x,manufacturer,price_x,subject_id_y,name_y,description_y,price_y,source_id,target_id,matching
0,207390654,Sony Handycam HDR-SR10 High Definition Digital...,16:9 - 2.7' Hybrid LCD,Sony,549.0,33161,Sony High Definition HDV Handycam Camcorder - ...,Sony High Definition HDV Handycam Camcorder - ...,NaN,33161,207390654,False
1,208085180,Pioneer DEH-2000MP Car Audio Player,"CD-RW - CD-Text, MP3, WMA, WAV - LCD - 4 - 200...",Pioneer,84.0,36258,D-Link Broadband Cable Modem - DCM202,D-Link Broadband Cable Modem - DCM202/ DOCSIS ...,79.00,36258,208085180,False
2,90125786,Sanus Wall/Ceiling Speaker Mount - WMS3S SILVER,Plastic - 8 lb,Sanus,NaN,17417,Sanus 13' - 30' VisionMount Flat Panel TV Silv...,Sanus 13' - 30' VisionMount Flat Panel TV Silv...,39.99,17417,90125786,False


### Connect to Neo4j

In [6]:
# connect to Neo4j
graph = Neo4jGraph(
    url=os.getenv('NEO4J_URL'),
    username=os.getenv('NEO4J_USER'),
    password=os.getenv('NEO4J_PASS')
)

# create database if does not exist
graph._database = "system"
query = f"CREATE DATABASE `{database}` IF NOT EXISTS"
graph.query(query)

# change to target database
graph._database = database
print("database:", graph._database)

# check nr nodes in the graph
graph.query("MATCH (n) RETURN count(n)")

database: abt-buy


[{'count(n)': 25884}]

### Error analysis

#### Analysis Errors on the training set

In [18]:
%%time

results = []

name_degree_threshold = 6
query = f"""
MATCH p1=(i1:Item)-[:HAS_NAME]->(n1:Name)-[:HAS_WORD]->(w:Word)<-[:HAS_WORD]-(n2:Name)<-[:HAS_NAME]-(i2:Item)
WHERE n1 <> n2
AND w.name_degree < {name_degree_threshold}
RETURN i1.subject_id, i2.subject_id"""
df_p = pd.DataFrame(graph.query(query))

if len(df_p) > 0:

    df_train_p = df_train.merge(df_p, left_on=['source_id', 'target_id'], right_on=['i1.subject_id', 'i2.subject_id'], how='left')
    df_train_p['p'] = df_train_p['i1.subject_id'] > 0

    prec, recall, fscore, support = precision_recall_fscore_support(df_train_p['matching'], df_train_p['p'], average='binary')

    results.append({'name_degree_threshold': name_degree_threshold, 'model': f'baseline_shared_word_threshold_{name_degree_threshold}', 'prec': prec, 'recall': recall, 'fscore': fscore})

df_results = pd.DataFrame(results).sort_values('fscore', ascending=False)
df_results

CPU times: user 139 ms, sys: 18.8 ms, total: 158 ms
Wall time: 229 ms


,name_degree_threshold,model,prec,recall,fscore
0,6,baseline_shared_word_threshold_6,0.835427,0.69055,0.756111


* Interestingly, fscore on the trainingset (0.756) is lower than on the validation set (0.789)


#### Mark errors in the graph

In [8]:
cond_y = (df_train_p['matching'] == True)
cond_p = (df_train_p['i1.subject_id'] > 0)

errors = {
    'tp': cond_y & cond_p,
    'fp': ~cond_y & cond_p,
    'tn': ~cond_y & ~cond_p,
    'fn': cond_y & ~cond_p
}

for error, cond_error in errors.items():
    df_train_p.loc[cond_error, 'error_train'] = error
    
df_train_p['error_train'].value_counts().to_frame().T

,tn,tp,fn,fp
error_train,4146,665,298,131


In [9]:
%%time
# Wall time: 24.8 s

run_cell = False
if run_cell == True:
    
    for i, (idx, row) in enumerate(df_train_p.iterrows()):
        if i % 250 == 0:
            print(f"{i}/{len(df_train_p)}")

        query = f"""
        MATCH (i1:Item)-[r]->(i2:Item) 
        WHERE i1.subject_id = {row['source_id']} 
        AND i2.subject_id={row['target_id']}
        MERGE (i1)-[:TRAIN_{row['error_train'].upper()}]->(i2)
        RETURN count(distinct r)
        """
        graph.query(query)

CPU times: user 14 µs, sys: 9 µs, total: 23 µs
Wall time: 16.9 µs


#### Examples of errors

<h4>Example of a FP</h4>
<ul>
    <li><i>Items share words "Plain" and "Fax/Copier" with name_degree 4</i></li>
    <li><i>Model IDS as very different: "KX-FP145" vs "KXFG2451"</i></li>    
</ul>
<img src="../images/3-example-train-fp.jpg">
<hr>
<h4>Example of FN</h4>
<ul><li><i>Model ID is different due to a separator "RDRVX560" vs "RDR-VX560"</i></li></ul>
<img src="../images/4.1-example-train-fn.jpg">
<h4>Example of FN</h4>
<ul>
    <li><i>Word matching is still case-sensitive! "Black" is not matched to "BLACK"</i></li>
    <li><i>Model IDS are written differently: "IH9BR" vs "IH9B6R"</i></li>
    </ul>
<img src="../images/4.2-example-train-fn.jpg">

### Add preprocessed Words

* Keep the original, but add (:WordLower) to see if performances increases

In [12]:
%%time
run_cell = False
if run_cell == True:
    query = "MATCH (w:Word) MERGE (ww:WordLower {value: toLower(w.value)}) MERGE (w)-[:TO_LOWER]->(ww)"
    graph.query(query)

    query = """
    MATCH (n)-[r:HAS_WORD]->(w:Word)-[:TO_LOWER]->(ww:WordLower)
    MERGE (n)-[:HAS_WORD_LOWER]->(ww)
    RETURN count(distinct r)"""
    graph.query(query)
    
    query = "MATCH p=(ww:WordLower)<-[:HAS_WORD_LOWER]-(n:Name) WITH ww, count(n) AS name_degree SET ww.name_degree = name_degree;"
    graph.query(query)

    query = "MATCH p=(ww:WordLower)<-[:HAS_WORD_LOWER]-(n:Description) WITH ww, count(n) AS description_degree SET ww.description_degree = description_degree;"
    graph.query(query)
    
    query = "MATCH (ww:WordLower) SET ww.display_value = ww.value + ' (' + ww.name_degree + ')' "
    graph.query(query)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 13.1 µs


<h4>Example of a FN with shared preprocessed words </h4>
<ul>
    <li><i>Left: Model ID can now be matched, this will correct the error</i></li>
    <li><i>Right: Very generic word, preprocessing will not correct the error</i></li></ul>        
</ul>
<img src="../images/4.4-example-train-fn-lowercase.jpg">


<h4>Example of a TN with shared preprocessed words </h4>
<ul>
    <li><i>The model claims this is TN because no rare words are shared (rarity: name_degree < 6)</i></li>
    <li><i>However, after pre-processing these items share the word "6-disc" which is only shared between these items (name_degree = 2)</i></li>
    <li><i>As these items should not be matched, this will result in a FP if you rerun the model with preprocessed words</i></li> 
</ul>
<img src="../images/5.example-train-tn-lowercase.jpg" width=600>




### Investigate items with conflicting groundtruth labels

<h4>Some items appear to have both True and False matching labels, this looks like a groundtruth error</h4>
<img src="../images/6-example-match-no-match.jpg" width="600">

In [14]:
query = "MATCH p=(i1:Item)-[:IS_MATCH]->(i2:Item)-[:NO_MATCH]-(i1) RETURN i1, i2"
results = graph.query(query)
results

[{'i1': {'subject_id': 33522, 'display_value': '33522 (abt)', 'source': 'abt'},
  'i2': {'display_value': '207986293 (buy)',
   'subject_id': 207986293,
   'source': 'buy'}},
 {'i1': {'subject_id': 22042, 'display_value': '22042 (abt)', 'source': 'abt'},
  'i2': {'display_value': '202238311 (buy)',
   'subject_id': 202238311,
   'source': 'buy'}}]

#### Check duplicates in the datasets

In [13]:
print("training set")
display(df_train[df_train.duplicated(['source_id', 'target_id'], keep=False)])

print("validation set")
display(df_val[df_val.duplicated(['source_id', 'target_id'], keep=False)])

print("test set")
display(df_test[df_test.duplicated(['source_id', 'target_id'], keep=False)])

training set


,source_id,target_id,matching
525,33522,207986293,True
881,22042,202238311,False
4666,22042,202238311,True
4786,33522,207986293,False


validation set


,source_id,target_id,matching
54,34130,207900174,False
780,34130,207900174,True


test set


,source_id,target_id,matching


* It seems that in both the train and validation set there are groundtruth errors.
* In the test set there are no duplicates

#### How would perfect prediction affect f1-score?

In [18]:
# get index of duplicated matches
idx = df_val[df_val.duplicated(['source_id', 'target_id'], keep=False)].index

# get max performance: use groundtruth to make predictions
df_val_p = df_val.copy()

# scenario 1: predict duplicated pair as a true match
df_val_p.loc[idx, 'matching'] = True
p = df_val_p['matching'].values*1
y = df_val['matching'].values*1
max_prec, max_recall, max_fscore, _ = precision_recall_fscore_support(y, p, average="binary")

print(f"predicting duplicated pair as a true match:")
print(f"- max precision: {max_prec:.4f}")
print(f"- max recall: {max_recall:.4f}")
print(f"- max fscore: {max_fscore:.4f}")
print()

# scenario 2: predict duplicated pair as no match
df_val_p.loc[idx, 'matching'] = False
p = df_val_p['matching'].values*1
y = df_val['matching'].values*1
max_prec, max_recall, max_fscore, _ = precision_recall_fscore_support(y, p, average="binary")

print(f"predicting duplicated pair as no match:")
print(f"- max precision: {max_prec:.4f}")
print(f"- max recall: {max_recall:.4f}")
print(f"- max fscore: {max_fscore:.4f}")

predicting duplicated pair as a true match:
- max precision: 0.9955
- max recall: 1.0000
- max fscore: 0.9977

predicting duplicated pair as no match:
- max precision: 1.0000
- max recall: 0.9955
- max fscore: 0.9977


* Maximum fscore on the validation set is 99.77%
* No duplicates on the test-set, so no effect on the public benchmark

### Improve baseline model

* Baseline: Items are a match if they share words with name_degree < 6
* New Model: Items are a match if they share _lowercase_ words with name_degree < 6

In [31]:
df_results_all = pd.read_csv(output_path / 'results.csv')

In [32]:
df_results_all

,model,threshold,prec,recall,fscore,evaluated_on
0,shared_word_threshold,6,0.885714,0.664286,0.759184,testset
1,shared_word_threshold,10,0.754717,0.759494,0.757098,testset
2,shared_word_threshold,7,0.803150,0.698630,0.747253,testset
3,shared_word_threshold,5,0.902174,0.628788,0.741071,testset
4,shared_word_threshold,15,0.635193,0.826816,0.718447,testset
5,shared_word_threshold,20,0.576667,0.878173,0.696177,testset
6,shared_word_threshold,4,0.954545,0.520661,0.673797,testset
7,shared_word_threshold,3,1.000000,0.474138,0.643275,testset
8,shared_word_threshold,30,0.474537,0.915179,0.625000,testset
9,shared_word_threshold,40,0.387358,0.937255,0.548165,testset


In [39]:
%%time

results = []

thresholds = np.concatenate([np.arange(8), np.arange(10, 20, 5), np.arange(20, 50, 10), np.arange(50, 150, 50)])[1:]
print("thresholds:", thresholds)

# Wall time: 29.3 s
for name_degree_threshold in thresholds:
    query = f"""
    MATCH p1=(i1:Item)-[:HAS_NAME]->(n1:Name)-[:HAS_WORD_LOWER]->(w:WordLower)<-[:HAS_WORD_LOWER]-(n2:Name)<-[:HAS_NAME]-(i2:Item)
    WHERE n1 <> n2
    AND w.name_degree < {name_degree_threshold}
    RETURN i1.subject_id, i2.subject_id"""
    df_p = pd.DataFrame(graph.query(query))

    if len(df_p) > 0:
    
        df_test_p = df_test.merge(df_p, left_on=['source_id', 'target_id'], right_on=['i1.subject_id', 'i2.subject_id'], how='left')
        df_test_p['p'] = df_test_p['i1.subject_id'] > 0

        prec, recall, fscore, support = precision_recall_fscore_support(df_test_p['matching'], df_test_p['p'], average='binary')

        results.append({'model': f'shared_word_lower_threshold', 'threshold': name_degree_threshold, 'prec': prec, 'recall': recall, 'fscore': fscore, 'evaluated_on': 'testset'})

df_results = pd.DataFrame(results).sort_values('fscore', ascending=False).reset_index(drop=True)
df_results

thresholds: [  1   2   3   4   5   6   7  10  15  20  30  40  50 100]
CPU times: user 12.2 s, sys: 1.51 s, total: 13.8 s
Wall time: 19.9 s


,model,threshold,prec,recall,fscore,evaluated_on
0,shared_word_lower_threshold,6,0.896226,0.673759,0.769231,testset
1,shared_word_lower_threshold,10,0.754601,0.763975,0.759259,testset
2,shared_word_lower_threshold,7,0.803030,0.711409,0.754448,testset
3,shared_word_lower_threshold,5,0.904255,0.634328,0.745614,testset
4,shared_word_lower_threshold,15,0.642553,0.829670,0.724221,testset
5,shared_word_lower_threshold,4,0.984848,0.532787,0.691489,testset
6,shared_word_lower_threshold,20,0.568627,0.874372,0.689109,testset
7,shared_word_lower_threshold,3,1.000000,0.482759,0.651163,testset
8,shared_word_lower_threshold,30,0.475556,0.926407,0.628488,testset
9,shared_word_lower_threshold,40,0.375918,0.948148,0.538381,testset


In [48]:
df_results_all = pd.concat([df_results, df_results_all]).sort_values('fscore', ascending=False).reset_index(drop=True).drop_duplicates()
df_results_all.to_csv(output_path / "results.csv", index=False)

In [49]:
df_results_all.head()

,model,threshold,prec,recall,fscore,evaluated_on
0,shared_word_lower_threshold,6,0.896226,0.673759,0.769231,testset
3,shared_word_lower_threshold,10,0.754601,0.763975,0.759259,testset
6,shared_word_threshold,6,0.885714,0.664286,0.759184,testset
7,shared_word_threshold,10,0.754717,0.759494,0.757098,testset
8,shared_word_lower_threshold,7,0.803030,0.711409,0.754448,testset


In [35]:
cond_y = (df_train_p['matching'] == True)
cond_p = (df_train_p['i1.subject_id'] > 0)

errors = {
    'tp': cond_y & cond_p,
    'fp': ~cond_y & cond_p,
    'tn': ~cond_y & ~cond_p,
    'fn': cond_y & ~cond_p
}

for error, cond_error in errors.items():
    df_train_p.loc[cond_error, 'error_train'] = error
    
df_train_p['error_train'].value_counts().to_frame().T

,tn,tp,fn,fp
error_train,4142,691,286,135
